In [ ]:
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, Model, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from keras.utils import to_categorical
from numpy.random import seed
from itertools import chain
from spacy import displacy
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow
import spacy
seed(1)
tensorflow.random.set_seed(2)


In [ ]:
data = pd.read_csv('ner_dataset.csv', encoding= 'unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


# Data Preparation for Neural Networks

In [ ]:
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}

    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))

    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [ ]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data_fillna = data.fillna(method='ffill', axis=0)
# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))

<ipython-input-4-f8b936d5d036>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(


In [ ]:
def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )

    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 6728 
train_tokens length: 6728 
test_tokens length: 997 
test_tags: 997 
val_tokens: 2243 
val_tags: 2243


# Training Neural Network

In [ ]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [ ]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))

    #Optimiser
    # adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
def train_model(X, y, model):
    loss = list()
    for i in tqdm(range(25)):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [ ]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 70, 64)            1897728   
                                                                 
 bidirectional_2 (Bidirecti  (None, 70, 128)           66048     
 onal)                                                           
                                                                 
 lstm_5 (LSTM)               (None, 70, 64)            49408     
                                                                 
 time_distributed_2 (TimeDi  (None, 70, 17)            1105      
 stributed)                                                      
                                                                 
Total params: 2014289 (7.68 MB)
Trainable params: 2014289 (7.68 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


  0%|          | 0/25 [00:00<?, ?it/s]

6/6 [==============================] - 41s 4s/step - loss: 2.3073 - accuracy: 0.7675 - val_loss: 0.6277 - val_accuracy: 0.9533


  4%|▍         | 1/25 [00:41<16:34, 41.44s/it]

6/6 [==============================] - 23s 4s/step - loss: 0.6110 - accuracy: 0.9525 - val_loss: 0.6005 - val_accuracy: 0.9533


...

 92%|█████████▏| 23/25 [13:03<00:51, 25.86s/it]

6/6 [==============================] - 21s 3s/step - loss: 0.2362 - accuracy: 0.9525 - val_loss: 0.2708 - val_accuracy: 0.9533


 96%|█████████▌| 24/25 [13:24<00:24, 24.35s/it]

6/6 [==============================] - 19s 3s/step - loss: 0.2306 - accuracy: 0.9525 - val_loss: 0.2552 - val_accuracy: 0.9533


100%|██████████| 25/25 [13:45<00:00, 33.01s/it]


# Testing the model

In [ ]:
nlp = spacy.load('en_core_web_sm')
text = nlp('Hi, My name is sid \n I am from United Kingdom \n I want to work with Google \n Steve Jobs is My Inspiration, seriously!')
displacy.render(text, style = 'ent', jupyter=True)